In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
tf.config.experimental.set_visible_devices([], 'GPU')
from  matplotlib import pyplot as plt

In [2]:
images = r"C:\Users\amr\images"

In [3]:
from keras.preprocessing.image import ImageDataGenerator
Batch_size = 128
train_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()
train_set = train_gen.flow_from_directory(images+"/train" , 
                                         color_mode="grayscale",
                                         batch_size=Batch_size,
                                         target_size = (48,48),
                                         class_mode="categorical",
                                          shuffle=True)

test_set = test_gen.flow_from_directory(images+"/validation" , 
                                         color_mode="grayscale",
                                         batch_size=Batch_size,
                                         target_size = (48,48),
                                         class_mode="categorical",
                                          shuffle=True)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [4]:
no_of_classes = 7
model = keras.Sequential([
    keras.layers.Conv2D(16 , (5,5) , padding="SAME" , activation="relu" , input_shape = [48,48,1]),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(32 , (3,3) , padding="SAME" , activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64 , (3,3) , padding="SAME" , activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128 , (3,3) , padding="SAME" , activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(200 , activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(100 , activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(no_of_classes , activation="softmax"),
    
])

In [5]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt , loss='categorical_crossentropy' , metrics=["accuracy"])

In [7]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

checkpoint = keras.callbacks.ModelCheckpoint(r"C:\Users\amr\Emotion Detection 2/model.h", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
reduce_learningrate = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,reduce_learningrate]

In [8]:
epochs = 48
model.fit_generator(generator=train_set , steps_per_epoch=train_set.n//train_set.batch_size,
                                epochs=epochs,
                                validation_data = test_set,
                                validation_steps = test_set.n//test_set.batch_size,
                                callbacks=callbacks_list)


Epoch 1/48


C:\Users\amr\AppData\Local\Temp\ipykernel_12796\848135606.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_set , steps_per_epoch=train_set.n//train_set.batch_size,


225/225 [==============================] - 71s 303ms/step - loss: 2.0493 - accuracy: 0.2103 - val_loss: 1.7966 - val_accuracy: 0.2746 - lr: 1.0000e-04
Epoch 2/48
225/225 [==============================] - 68s 303ms/step - loss: 1.8329 - accuracy: 0.2464 - val_loss: 1.7323 - val_accuracy: 0.2996 - lr: 1.0000e-04
Epoch 3/48
225/225 [==============================] - 70s 309ms/step - loss: 1.7916 - accuracy: 0.2631 - val_loss: 1.7068 - val_accuracy: 0.3081 - lr: 1.0000e-04
Epoch 4/48
225/225 [==============================] - 68s 304ms/step - loss: 1.7627 - accuracy: 0.2769 - val_loss: 1.6836 - val_accuracy: 0.3338 - lr: 1.0000e-04
Epoch 5/48
225/225 [==============================] - 68s 303ms/step - loss: 1.7402 - accuracy: 0.2908 - val_loss: 1.6597 - val_accuracy: 0.3335 - lr: 1.0000e-04
Epoch 6/48
225/225 [==============================] - 70s 310ms/step - loss: 1.7129 - accuracy: 0.3077 - val_loss: 1.6368 - val_accuracy: 0.3506 - lr: 1.0000e-04
Epoch 7/48
225/225 [===================

In [9]:
model.evaluate(test_set)

56/56 [==============================] - 3s 48ms/step - loss: 1.1644 - accuracy: 0.5601


[1.1644164323806763, 0.5601471662521362]